In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
import import_ipynb
import notebook2script_py

importing Jupyter notebook from notebook2script_py.ipynb


In [5]:
notebook2script_py.notebook2script('_05_anneal (Skipped).ipynb')

Converted _05_anneal (Skipped).ipynb to nb_05.py


In [6]:
import torch
import matplotlib.pyplot as plt
from exp.nb_05 import *

# Creating your own callback


In [7]:
from time import sleep

In [10]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i * i
        sleep(1)
        
    return res

In [11]:
slow_calculation()

30

In [23]:
def slow_calculation(cb = None):
    res = 0
    for i in range(5):
        res += i * i
        sleep(1)
        if cb: cb(i)
    
    return res

In [24]:
def show_progress(epoch):
    print(f"Awesome! We've finished epoch {epoch}!")

In [25]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

# Lambdas and partials

In [26]:
slow_calculation(lambda o : print(f"Awesome! We've finished epoch {o}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

In [27]:
def show_progress(exclamation , epoch):
    print(f"{exclamation}! We've finished epoch {epoch}!")

In [28]:
slow_calculation(lambda o: show_progress("OK I guess" , o))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [29]:
def make_show_progress(exclamation):
    def _inner(epoch):
        print(f"{exclamation}! we've finishd epoch {epoch}!")
    return _inner

In [30]:
slow_calculation(make_show_progress("Nice!"))

Nice!! we've finishd epoch 0!
Nice!! we've finishd epoch 1!
Nice!! we've finishd epoch 2!
Nice!! we've finishd epoch 3!
Nice!! we've finishd epoch 4!


30

In [31]:
f2 = make_show_progress("Terrific")

In [32]:
slow_calculation(f2)

Terrific! we've finishd epoch 0!
Terrific! we've finishd epoch 1!
Terrific! we've finishd epoch 2!
Terrific! we've finishd epoch 3!
Terrific! we've finishd epoch 4!


30

In [33]:
from functools import partial

In [34]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [35]:
f2 = partial(show_progress, "OK I guess")

# Callbacks as callable classes


In [39]:
class ProgressShowingCallback():
    def __init__(self , exclamation = "Awesome"):
        self.exclamation = exclamation
    
    def __call__(self , epoch):
        print(f"{self.exclamation}! we've finished epoch {epoch}!")

In [40]:
cb = ProgressShowingCallback("Just super")

In [41]:
slow_calculation(cb)

Just super! we've finished epoch 0!
Just super! we've finished epoch 1!
Just super! we've finished epoch 2!
Just super! we've finished epoch 3!
Just super! we've finished epoch 4!


30

# Multiple callback funcs; *args and **kwargs

In [44]:
def f(*args , **kwargs): print(f"args: {args}; kwargs: {kwargs}")

In [45]:
f(3 , 'a', thing1 = "hello")

args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [46]:
def slow_calculation(cb = None):
    res = 0
    for i in range(5):
        if cb:
            cb.before_calc(i)
            
        res += i * i
        sleep(1)
        if cb:
            cb.after_calc(i , val = res)
            
    return res

In [47]:
class PrintStepCallback():
    
    def __init__(self):
        pass
    
    def before_calc(self , *argw , **kwargs):
        print(f"About to start")
        
    def after_calc(self , *args , **kwargs):
        print(f"Done step")

In [48]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [49]:
class PrintStatusCallback():
    def __init__(self): 
        pass
    
    def before_calc(self, epoch, **kwargs): 
        print(f"About to start: {epoch}")
        
    def after_calc (self, epoch, val, **kwargs): 
        print(f"After {epoch}: {val}")

In [50]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

# Modifying behavior

In [52]:
def slow_calculation(cb = None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb , 'before_calc'): 
            cb.before_calc(i)
        res += i * i
        sleep(0)
        if cb and hasattr(cb , 'after_calc'):
            if cb.after_calc(i , res):
                print("stopping early")
                break
                
    return res

In [53]:
class PrintAfterCallback():
    def after_calc(self , epoch , val):
        print(f"After {epoch}: {val}")
        if val > 10: 
            return True

In [54]:
slow_calculation(PrintAfterCallback())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

In [83]:
class SlowCalculator():
    def __init__(self , cb = None):
        self.cb , self.res = cb , 0
        
    def callback(self , cb_name , *args):
        if not self.cb:
            return 
        cb = getattr(self.cb , cb_name , None)
        if cb:
            return cb(self, *args)
        
    def calc(self):
        for i in range(5):
            self.callback('before_calc' , i)
            self.res += i * i
            sleep(1)
            if self.callback('after_calc' , i):
                print('stopping early')
                break

In [84]:
class ModifyingCallback():
    def after_calc(self , calc , epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res > 10: 
            return True
        if calc.res < 3: 
            calc.res = calc.res * 2

In [85]:
calculator = SlowCalculator(ModifyingCallback())

In [86]:
calculator.calc()
calculator.res

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


15

# Reference for this notebook

In [1]:
#https://github.com/fastai/course-v3/blob/master/nbs/dl2/05a_foundations.ipynb